In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

# Setting up the send email tool

In [2]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SendEmailInput(BaseModel):
    email: str = Field(description="The email text to be sent")

class SendEmailTool(BaseTool):
    name = "email_triage"
    description = "useful for when you need to send an email to someone"
    args_schema: Type[BaseModel] = SendEmailInput

    def _run(
        self, email: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        print(email)
        return "Email sent successfully"

    async def _arun(
        self, email: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        print(email)
        return "Email sent successfully"

# Setting up the fetch transaction tool

In [3]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class FetchTrxInput(BaseModel):
    reference_number: str = Field(description="The transaction reference number")

class FetchTransactionTool(BaseTool):
    name = "fetch_transactions"
    description = "useful for when you want to fetch the transaction details given reference number"
    args_schema: Type[BaseModel] = FetchTrxInput

    def _run(
        self, reference_number: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return "The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the account"

    async def _arun(
        self, reference_number: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        return "The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the account"

# Setup a retriver db

In [4]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./rag_document.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

/Users/vizsatiz/Documents/hub/flo/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
from flo_ai.retrievers.flo_retriever import FloRagBuilder
from flo_ai.retrievers.flo_compression_pipeline import FloCompressionPipeline

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
session = FloSession(llm)
builder = FloRagBuilder(session, db.as_retriever())
compression_pipeline = FloCompressionPipeline(OpenAIEmbeddings(model="text-embedding-3-small"))
compression_pipeline.add_embedding_reduntant_filter()
compression_pipeline.add_embedding_relevant_filter()

retriever_tool = builder.with_multi_query().with_compression(compression_pipeline).build_retriever_tool(name="HousingLoanRetreiver",
                                                                                               description="Tool to fetch data around housing loans")
session.register_tool(name="HousingLoanTool", tool=retriever_tool)

# Register the tools

In [6]:
session.register_tool(
    name="SendEmailTool", 
    tool=SendEmailTool()
).register_tool(
    name="FetchTransactionTool",
    tool=FetchTransactionTool()
)

# Setting up the agentic flo

In [14]:
agent_yaml = """
apiVersion: flo/alpha-v1
kind: FloRoutedTeam
name: support-email-handler
team:
    name: SupportTicketHandler
    router:
        name: SupportSupervisor
        kind: supervisor
    agents:
      - name: EmailSender
        prompt: You are capable of sending the reply email but constructing a apt response
        tools:
          - name: SendEmailTool
      - name: TransactionFetcher
        prompt: You are capable of fetching any kind of transactions from the database given transaction reference id
        tools:
          - name: FetchTransactionTool
      - name: HousingLoanTeamLead
        prompt: Fetch the housing loan information from the db and answer the question
        tools:
          - name: HousingLoanTool
"""

# Setup input prompt

1. This works if the transaction number is passed
2. Asks for transaction number if its missing

In [18]:
email_input_for_transaction_without_id = """
Hi Tom,

We have a failed transaction. Can you tell me more about why this failed.

Thanks
Vishnu
"""

email_input_for_transaction_with_id = """
Hi Tom,

We have a failed transaction. Can you tell me more about why this failed.

Transaction ID: 12123123432

Thanks
Vishnu
"""

email_input_for_housing_loan = """
Hi Tom,

I am looking for a housing loan, is the interest percentage on the loan

The tenure is 8 years
Amount am looking for is 85 lakhs
And am looking for fixed rate housing loan

Thanks
Vishnu
"""

In [19]:
input_prompt = """
You are a support ticket handling agent. You are expected to understand the email, and perform the following steps:

1. Reply asking for more information if there is missing information in the email
2. Perform the neccessary steps to required to answer the users question and reply the user query.
3. If are not able to come up with a answer just reply I dont know over the email

Email:

""" + email_input_for_housing_loan

## Run the flo

In [20]:
flo: Flo = Flo.build_with_yaml(session, yaml=agent_yaml)
for s in flo.stream(input_prompt):
     if "__end__" not in s:
        print(s)
        print("----")

{'SupportSupervisor-I65cJ': {'next': 'HousingLoanTeamLead-EfugX'}}
----


> Entering new AgentExecutor chain...

Invoking: `HousingLoanRetreiver` with `{'query': 'fixed rate housing loan, 8 years tenure, 85 lakhs'}`


Maximum funding for your dream house -85% of the project cost.
Loan amount up to Rs. 1500 Lakhs 
Longer repayment period up to 360 months
Repayment holiday up to 36 months.
Loan can be availed in overdraft format
Minimal paperwork
Speedy loan approval
No pre closure charges
Low processing fees
Optional funding for life insurance 
Easy top up loans in future.
Repayment of the loan is made as Equated Monthly Installments (EMI).


Eligibility
For Residents:

 

For NRIs:

Housing Loan

Your dream home is never far away! Get hassle free home loans from Federal Bank to turn your dream home into reality. We assist you to realize your dream home. Avail your Housing Loan from us at competitive interest rates. The loan scheme assists borrowers for construction of house, acquisitio